In [6]:
import multiprocessing.pool
from datetime import datetime
import pandas as pd
import os
import numpy as np
from functools import partial
import dask.dataframe as dd
import dask.multiprocessing
LOB = pd.DataFrame()

In [7]:
laCie = '/media/ak/LaCie/'
trades_quotes_folder = os.path.join(laCie, 'Trades and Quotes Purchased July 2020/')
processed_sampled_data = os.path.join(laCie, 'ProcessedSampledData/')
symbols = os.listdir(trades_quotes_folder)
symbol = 'FLG'
symbol_trades_quotes_folder = os.path.join(trades_quotes_folder, symbol)

In [8]:
symbol_files = os.listdir(symbol_trades_quotes_folder)


In [9]:
fileidx = 1
idx_symbol_file = os.path.join(symbol_trades_quotes_folder, symbol_files[fileidx])
print(idx_symbol_file)

/media/ak/LaCie/Trades and Quotes Purchased July 2020/FLG/FLGM2019.csv


In [10]:
df = pd.read_csv(idx_symbol_file)



In [11]:
### conversion of Dates

In [12]:
inputDates = df.Date
date_time_str = inputDates[1]
date_time_str

'05/01/2019'

In [13]:
new_date_time = datetime.strptime(date_time_str, "%d/%m/%Y").strftime("%Y%m%d")

In [14]:
new_date_time #new format for the date

'20190105'

In [15]:
df_per_date = df[df['Date'] == date_time_str] # dataframe filtered for the new format

In [16]:
type(df_per_date['Time'][0])

str

In [17]:
#qt_tmp_df['TradeTime'] = pd.to_datetime(qt_tmp_df.time).values
time_obj =pd.to_datetime(df_per_date.Time)

In [18]:
type(time_obj)

pandas.core.series.Series

In [19]:
# pd.to_timedelta(df_per_date.Time).seconds


In [20]:
# qt_tmp_df['TradeTime'] = pd.to_datetime(qt_tmp_df.time).values
# dfc['Time_of_Sail'] = pd.to_datetime(dfc['Time_of_Sail'],format= '%H:%M:%S' ).dt.time

In [21]:
df_per_date['TradeTime']  = pd.to_datetime(df_per_date.Time).dt.time

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
# df_per_date['Date'] = datetime.strptime(df_per_date.Date, "%d/%m/%Y").strftime("%Y%m%d")

In [23]:
bidPrice = df_per_date.filter(like = '-BidPrice')
bidSize = df_per_date.filter(like = '-BidSize')


In [24]:
bidPrice['L1-BidPrice']

0        127.33
1        127.33
2        127.33
3        127.33
4        127.33
          ...  
64032    127.30
64033    127.30
64034    126.42
64035       NaN
64036       NaN
Name: L1-BidPrice, Length: 64037, dtype: float64

In [25]:
LOB['BidPrice'] =(bidSize*bidPrice.values).sum(axis=1)/bidSize.sum(axis=1)
LOB['AskPrice'] =(df_per_date.filter(like = '-AskSize')*df_per_date.filter(like = '-AskPrice').values).sum(axis=1)/df_per_date.filter(like = '-AskSize').sum(axis=1)


In [26]:
df_per_date

,Date,Time,L1-BidPrice,L1-BidSize,L1-BuyNo,L1-AskPrice,L1-AskSize,L1-SellNo,L2-BidPrice,L2-BidSize,...,L9-AskPrice,L9-AskSize,L9-SellNo,L10-BidPrice,L10-BidSize,L10-BuyNo,L10-AskPrice,L10-AskSize,L10-SellNo,TradeTime
0,05/01/2019,08:00:00.169284,127.33,59.0,8.0,127.35,2.0,1.0,127.32,38.0,...,127.43,38.0,9.0,127.24,26.0,10.0,127.44,27.0,6.0,08:00:00.169284
1,05/01/2019,08:00:00.171387,127.33,59.0,8.0,127.35,7.0,2.0,127.32,41.0,...,127.43,41.0,10.0,127.24,29.0,11.0,127.44,30.0,7.0,08:00:00.171387
2,05/01/2019,08:00:00.178396,127.33,59.0,8.0,127.34,19.0,3.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.178396
3,05/01/2019,08:00:00.190400,127.33,59.0,8.0,127.34,19.0,4.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.190400
4,05/01/2019,08:00:00.205580,127.33,64.0,9.0,127.34,19.0,4.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.205580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64032,05/01/2019,18:00:00.128145,127.30,6.0,1.0,NaN,NaN,NaN,127.25,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64033,05/01/2019,18:00:00.128145,127.30,6.0,1.0,NaN,NaN,NaN,126.42,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64034,05/01/2019,18:00:00.128145,126.42,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64035,05/01/2019,18:00:00.128145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145


In [28]:
LOB['AskSize'] = (df_per_date.filter(like = '-AskSize')*df_per_date.filter(like = '-SellNo').values).sum(axis=1)/df_per_date.filter(like = '-SellNo').sum(axis=1)

In [29]:
LOB['BidSize'] = (df_per_date.filter(like = '-BidSize')*df_per_date.filter(like = '-BuyNo').values).sum(axis=1)/df_per_date.filter(like = '-BuyNo').sum(axis=1)

In [30]:
LOB

,BidPrice,AskPrice,AskSize,BidSize
0,127.298333,127.406070,30.268293,30.966102
1,127.296296,127.404533,32.450980,31.823529
2,127.296352,127.396295,31.957447,32.057971
3,127.296352,127.396295,31.687500,32.057971
4,127.297028,127.396630,32.170213,33.085714
...,...,...,...,...
64032,127.224762,NaN,NaN,7.000000
64033,127.174286,NaN,NaN,3.500000
64034,126.420000,NaN,NaN,1.000000
64035,NaN,NaN,NaN,NaN
